In [1]:
!pip install opencc-python-reimplemented
!wget https://dumps.wikimedia.org/zhwiki/20200301/zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2


# 用 Gensim 對中文維基百科語料上 word2Vec 相似度計算實驗
from gensim.corpora import WikiCorpus
# 使用檔案名稱
wiki_corpus = WikiCorpus('zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2', dictionary={})
next(iter(wiki_corpus.get_texts()))[:10]

text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))
# 引入 Jieba 中文分詞
import jieba
# 中文繁簡轉換
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
train_data = cc.convert(train_data)
train_data = jieba.lcut(train_data)
train_data = [word for word in train_data if word != '']
train_data = ' '.join(train_data)
open('seg.txt', 'w', encoding='utf-8').write(train_data)
# 訓練中文詞向量
from gensim.models import word2vec


# Settings
seed = 666
sg = 0
window_size = 10
vector_size = 100
min_count = 1
workers = 8
epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')


from gensim.models import word2vec

string = '微生物'
model = word2vec.Word2Vec.load('word2vec.model')
print(string)

for item in model.wv.most_similar(string):
    print(item)

     |████████████████████████████████| 491kB 9.2MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.5-py2.py3-none-any.whl size=485664 sha256=9a910a12bc855386bf16c5dd63222219e8ca40e594c5d19caeca985b6ecc5e78
  Stored in directory: /root/.cache/pip/wheels/36/a0/10/888b9ac7f10154caaa6a73270b1f085e0a7b241baa672bbe49
Successfully built opencc-python-reimplemented
--2020-04-29 21:49:35--  https://dumps.wikimedia.org/zhwiki/20200301/zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172586252 (165M) [application/octet-stream]
Saving to: ‘zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2’

zhwiki-20200301-pag 100%[===================>] 164.59M  4.84MB/s    in 35s     

2020-04-29 21:50:11 (4.6

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.757 seconds.
Prefix dict has been built successfully.
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


微生物
('細菌', 0.8879451751708984)
('菌', 0.8650882840156555)
('代謝', 0.8611363172531128)
('真菌', 0.8601622581481934)
('致病', 0.8497619032859802)
('藻類', 0.8496725559234619)
('病理', 0.8446841835975647)
('免疫', 0.842602550983429)
('胰島素', 0.8363394737243652)
('毒素', 0.8325620293617249)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
